# Visualising Word Vectors Using TF2 [Advisable]
>Exploration and Visualisation of Word Vectors Using TensorFlow 2

- toc: false 
- badges: true
- comments: true
- categories: [jupyter,tensorflow,python]

Tensorflow released their latest version of <a href="https://www.youtube.com/watch?v=k5c-vg4rjBw">TensorFlow 2</a> on September 30, 2019.

As also mentioned in <font color='red'>tb1vis.ipynb</font> the reason we are visualising FastText is because:

<blockquote>
FastText uses the concept that embeddings are formed based on the sub-word approach, this method helps us to visualise and obtain misspellings of a word or different spellings of the same word.</blockquote>

I couldn't find any blogs on the Internet that have covered or updated their code which describes the visualisation of embeddings through the latest version of Tensorflow. 

Although I did take the help of an issue <a href="https://github.com/tensorflow/tensorboard/issues/2471 ">TF 2.0 API for using the embedding projector</a> raised on the Tensorflow repository and have come to a concluding notebook suiting my goal i.e. to visualise FastText embeddings using TF2.  

The tensorflow version used in this notebook is version 2.

![](my_icons/tf2.png)

In [1]:
# import statements
from pathlib import PurePath

import fasttext
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

from tensorboard.plugins import projector
from tensorboard.plugins.projector import ProjectorConfig

In [2]:
#hide_output

# load pre-trained fasttext model
model = fasttext.load_model("fasttextmodel.bin")

In [3]:
for i, w in enumerate(model.get_words()):
    print(w)
    if i > 4:
        break

s
said
mr
</s>
people
new


In [4]:
#hide_output

# number of words in the dataset
VOCAB_SIZE = len(model.get_words())


# size of the dimension of each word vector
EMBEDDING_DIM = len(model.get_word_vector(w))


# 2D numpy array initialised to store words with their vector representation
embed = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
embed.shape


(10891, 300)

In [5]:
# store the vector representation of each word in the 2D numpy array
for i, word in enumerate(model.get_words()):
    embed[i] = model.get_word_vector(word)
embed


array([[-0.11363645,  0.00304414,  0.00589875, ...,  0.00278742,
         0.03564256, -0.10496949],
       [ 0.05821591,  0.07343163, -0.06941246, ...,  0.00737938,
         0.08668958, -0.05127012],
       [ 0.06867523, -0.02112868, -0.02132288, ...,  0.05362611,
         0.13982825,  0.04221647],
       ...,
       [ 0.16511762,  0.04439345, -0.14276202, ...,  0.02632121,
         0.03970968,  0.03706815],
       [ 0.09471416,  0.09356211,  0.00358974, ..., -0.0174412 ,
         0.13414964,  0.02268019],
       [ 0.07753251, -0.02356024, -0.05303693, ...,  0.14130574,
         0.09740689,  0.0418443 ]])

In [6]:
# path to store the words
tsv_file_path = "tensorboard/metadata.tsv"


In [8]:
#hide output
with open(tsv_file_path, "w+", encoding="utf-8") as f:
    for i, word in enumerate(model.get_words()):
        f.write(word + "\n")  # write the words to an external file
embed.shape

(10891, 300)

---

### Projection on Tensorboard 2
                                  

Steps for projection:
 
1. Define the function `register_embedding()` and `save_label_tsv()` to configure the projector as well as save the projector configuration files and metadata file to the same folder.
2. Initialise the path variables accordingly and call the above function with suitable path variables as shown in below cells.
3. Creation of the tensorflow variable instead of tensorflow placeholder.
4. A saver class object is initialised and checkpoint is created. 

---

### Differences between TF 1 and TF 2
<ul>
<li>We cannot call the reset default graph method directly from tf library which we did for TensorFlow 1. It is invoked in TF2 as: 

```python
from tensorflow.python.framework import ops
ops.reset_default_graph()
```
</li>

<li>There is no placeholder required here. A tf variable is created which is passed parameters. A TF variable is  shown below. Here the parameters are x: the array which contains the embeddings and name: name of the embedding file.

`python
tensor_embeddings = tf.Variable(x, name=EMBEDDINGS_TENSOR_NAME)
`
According to the TF2 documentation a <a href="https://www.tensorflow.org/guide/variable">tensorflow variable</a> is defined as:

<blockquote>
A variable maintains shared, persistent state manipulated by a program.

  The Variable() constructor requires an initial value for the variable, which
  can be a Tensor of any type and shape. This initial value defines the type
  and shape of the variable. After construction, the type and shape of the
  variable are fixed. The value can be changed using one of the assign methods.
</blockquote>
</li>

<li>There is no concept of `session` as well as `saver` in TF2 yet. To workaround this, we just use the `saver` class for the creation of checkpoints by initialising the `saver` object with the tensorflow variable and pass `None` as value to the `session` parameter in `saver.save()`.

```python
saver = tf.compat.v1.train.Saver([tensor_embeddings])  
saver.save(sess=None, global_step=STEP, save_path=EMBEDDINGS_FPATH)
```
</ul>
                           

---

In [10]:
ops.reset_default_graph()  # clearing the default graph stack


def register_embedding(
    embedding_tensor_name: str, meta_data_fname: str, log_dir: str,
) -> None:

    """
    Configuring the projector to be read by the tensorboard.
    
    Args:
    embedding_tensor_name(str): embeddings file name
    meta_data_fname(str): metadata file name
    log_dir(str): folder where tensorboard files and the metadata file are saved
    
    Returns:
    None    
    
    """
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_tensor_name
    embedding.metadata_path = meta_data_fname
    projector.visualize_embeddings(
        log_dir, config
    )  # storing the configuration files of projector where tensorboard files are saved

In [13]:
def save_labels_tsv(labels: list, filepath: str, log_dir: str,) -> None:

    """
    Storing the vocabulary of words in the dataset to a file
    
    Args:
    labels: vocabulary i.e. words in the dataset
    filepath: metadata file name
    log_dir: "folder where tensorboard files and projector files are saved
    
    Returns:
    None  
    
    """

    with open(PurePath(log_dir, filepath), "w") as f:
        for label in labels:
            f.write("{}\n".format(label))

In [14]:
LOG_DIR = "tb2files"  # folder which will contain all the tensorboard log files

# Labels i.e. the words in the dataset will be stored in this file
META_DATA_FNAME = "meta.tsv"

# name of the file which will have the embeddings stored
EMBEDDINGS_TENSOR_NAME = "embeddings"

# path for checkpoint of the saved embeddings
EMBEDDINGS_FPATH = PurePath(LOG_DIR, EMBEDDINGS_TENSOR_NAME + ".ckpt")
STEP = 0


x = embed  # array containing the embeddings
y = model.get_words()  # list containing the vocabulary
register_embedding(EMBEDDINGS_TENSOR_NAME, META_DATA_FNAME, LOG_DIR)
save_labels_tsv(y, META_DATA_FNAME, LOG_DIR)

In [15]:
tensor_embeddings = tf.Variable(
    x, name=EMBEDDINGS_TENSOR_NAME
)  # creation of the tensorflow variable, x: array which contains the embeddings,
# name: name of the file which will have the embeddings stored

In [16]:
#hide_output

saver = tf.compat.v1.train.Saver(
    [tensor_embeddings]
)  # Tensorflow variable passed as argument for saver object to be initialised
saver.save(
    sess=None, global_step=STEP, save_path=EMBEDDINGS_FPATH
)  # saving the checkpoint for the embedding files

'tb2files/embeddings.ckpt-0'

---